In [25]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import mlflow
from mlflow.models import infer_signature

In [26]:
print(f'Numpy Version: {np.__version__}')
print(f'Pandas Version: {pd.__version__}')


Numpy Version: 2.2.1
Pandas Version: 2.2.3


In [27]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')


In [28]:
## Load the dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define model hyperparameters
params = {
    'penalty':'l2',
    'solver': 'lbfgs',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state':8888
}


## Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)



e:\AI & Data Scientist\AIMLEngineer\mlflow\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [29]:
## Predict on the test set
y_pred = lr.predict(X_test)
print(y_pred)

[2 2 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 2 1 0 0 1 0 1 1 2 1 1 2]


In [30]:
accuracy = accuracy_score(y_test, y_pred)

In [31]:
accuracy

1.0

In [32]:
## MLFlow tracking
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')

## Create a new MLflow experiment
mlflow.set_experiment('MLFlow Quickstart')

with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params=params)

    ## Log the accuracy metrics
    mlflow.log_metric('accuracy', accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag('Training info', 'Basic LR Model for Iris dataset')

    ## Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr, 
        artifact_path='iris_model', 
        signature=signature, 
        input_example=X_train,
        registered_model_name="tracking_quickstart",
    )




2025/01/18 23:31:54 INFO mlflow.tracking.fluent: Experiment with name 'MLFlow Quickstart' does not exist. Creating a new experiment.


Successfully registered model 'tracking_quickstart'.
2025/01/18 23:31:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_quickstart, version 1


🏃 View run shivering-ray-917 at: http://127.0.0.1:5000/#/experiments/375223906825683941/runs/b69a51cd0a20435183a9601f8dcc028b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/375223906825683941


Created version '1' of model 'tracking_quickstart'.


### Experiment 2

In [33]:
# Define model hyperparameters
params = {
    'penalty':'l2',
    'solver': 'newton-cg',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state':8888
}


## Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

e:\AI & Data Scientist\AIMLEngineer\mlflow\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888,
                   solver='newton-cg')

In [34]:
## Predict on the test set
y_pred = lr.predict(X_test)
print(y_pred)

[2 2 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 2 1 0 0 1 0 1 1 2 1 1 2]


In [35]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

In [36]:

with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params=params)

    ## Log the accuracy metrics
    mlflow.log_metric('accuracy', accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag('Training info', 'Basic LR Model for Iris dataset')

    ## Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr, 
        artifact_path='iris_model', 
        signature=signature, 
        input_example=X_train,
        registered_model_name="tracking_quickstart",
    )


Registered model 'tracking_quickstart' already exists. Creating a new version of this model...
2025/01/18 23:32:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_quickstart, version 2


🏃 View run gentle-stoat-41 at: http://127.0.0.1:5000/#/experiments/375223906825683941/runs/3d7336d45e4749d290956524139474a8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/375223906825683941


Created version '2' of model 'tracking_quickstart'.


# Inferencing and validating my model

In [37]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/b69a51cd0a20435183a9601f8dcc028b/iris_model'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      5.6,
      2.9,
      3.6,
      1.3
    ],
    [
      5.8,
      2.7,
      3.9,
      1.2
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      5.0,
      3.4,
      1.6,
      0.4
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      5.5,
      2.4,
      3.7,
      1.0
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      5.5,
      2.3,
      4.0,
      1.3
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      6.7,
      2.5,
      5.8,
      1.8
    ],
    [
      7.9,
      3.8,
      6.4,
      2.0
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      6.1,
      2.9,
      4.7,
      1.4
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      6.4,
      2.7,
      5.3,
      1.9
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      6.0,
      2.9,
      4.5,
      1.5
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      4.8,
      3.4,
      1.6,
      0.2
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      7.7,
      2.6,
      6.9,
      2.3
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      6.8,
      3.2,
      5.9,
      2.3
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      6.7,
      3.0,
      5.0,
      1.7
    ],
    [
      5.6,
      2.5,
      3.9,
      1.1
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      6.3,
      2.7,
      4.9,
      1.8
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      6.8,
      2.8,
      4.8,
      1.4
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      7.3,
      2.9,
      6.3,
      1.8
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      6.7,
      3.1,
      5.6,
      2.4
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.5,
      3.2,
      5.1,
      2.0
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      6.1,
      2.8,
      4.7,
      1.2
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      5.4,
      3.4,
      1.5,
      0.4
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([2, 2, 2, 0, 1, 2, 0, 1, 1, 2, 1, 0, 0, 0, 2, 0, 0, 2, 1, 1, 1, 0,
       1, 1, 1, 2, 0, 2, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 2, 2, 0, 1, 0, 0,
       2, 0, 1, 2, 1, 1, 0, 2, 0, 2, 0, 1, 0, 0, 1, 2, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 1, 2, 0, 2, 2,
       2, 0, 2, 2, 0, 0, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 2, 2, 1, 0, 0, 2,
       0, 0, 1, 2, 0, 2, 2, 0, 1, 2])

#### Load the model back for prediction as a generic python function model

In [38]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features_name)
result['actual_class']=y_test
result['predicted_class']=predictions

In [40]:
result[:10]

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                6.7               3.0                5.2               2.3   
1                6.4               2.8                5.6               2.2   
2                5.5               2.6                4.4               1.2   
3                6.4               3.2                4.5               1.5   
4                6.3               3.3                4.7               1.6   
5                4.7               3.2                1.3               0.2   
6                4.6               3.1                1.5               0.2   
7                5.1               3.5                1.4               0.2   
8                6.0               2.2                4.0               1.0   
9                4.9               2.4                3.3               1.0   

   actual_class  predicted_class  
0             2                2  
1             2                2  
2             1                1  
3             1                1  
4             1                1  
5             0                0  
6             0                0  
7             0                0  
8             1                1  
9             1                1

##  Model Registry

The MlFlow Model Registry is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLFlow Model. It provides model lineage (which MLFlow experiement and run produced the model), model versioning, model aliasing, model tagging, and annotations.

In [41]:

## Create a new MLflow experiment
mlflow.set_experiment('MLFlow Quickstart')

with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params=params)

    ## Log the accuracy metrics
    mlflow.log_metric('accuracy', accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag('Training info2', 'Basic LR Model for Iris dataset')

    ## Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr, 
        artifact_path='iris_model', 
        signature=signature, 
        input_example=X_train
    )


🏃 View run likeable-bat-158 at: http://127.0.0.1:5000/#/experiments/375223906825683941/runs/eca9a881875542479fb06584337d6084
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/375223906825683941


In [47]:
# Inferenceing from model registry
import mlflow.sklearn

model_name='tracking_quickstart'
model_version='latest'

model_uri=f'models:/{model_name}/{model_version}'

model = mlflow.sklearn.load_model(model_uri)

model


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888,
                   solver='newton-cg')

In [48]:
model_uri

'models:/tracking_quickstart/latest'

In [49]:
y_pred_new=model.predict(X_test)
print(y_pred_new)

[2 2 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 2 1 0 0 1 0 1 1 2 1 1 2]
